<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [4]:
df = df.sample(False, 0.5)

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

In [6]:
pandas_df = df.limit(20).toPandas()

pandas_df.loc[:, :'_c13']

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13
0,1,0.0,-1,NaN,NaN,1465.0,0.0,17.0,0,4.0,0.0,4.0,NaN,NaN
1,0,8.0,0,15.0,20.0,115.0,24.0,8.0,23,24.0,2.0,2.0,NaN,20.0
2,0,NaN,5,30.0,4.0,NaN,NaN,0.0,5,4.0,NaN,0.0,NaN,5.0
3,0,NaN,7,1.0,NaN,9128.0,NaN,0.0,0,7.0,NaN,0.0,NaN,NaN
4,0,0.0,18,15.0,9.0,4494.0,NaN,0.0,9,8.0,0.0,0.0,NaN,9.0
5,0,NaN,19,32.0,0.0,1994.0,NaN,0.0,19,26.0,NaN,0.0,NaN,3.0
6,0,0.0,76,3.0,NaN,5029.0,NaN,NaN,16,NaN,0.0,NaN,NaN,NaN
7,0,0.0,1,51.0,11.0,20692.0,1645.0,3.0,21,1580.0,0.0,1.0,NaN,11.0
8,0,NaN,19,31.0,8.0,7130.0,126.0,1.0,20,116.0,NaN,1.0,NaN,8.0
9,0,NaN,2242,13.0,7.0,11786.0,63.0,29.0,13,87.0,NaN,1.0,NaN,13.0


In [7]:
pandas_df.loc[:, '_c14':'_c26']

,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22,_c23,_c24,_c25,_c26
0,241546e0,38a947a1,fa673455,6a14f9b9,25c83c98,fe6b92e5,1c86e0eb,1f89b562,a73ee510,e7ba2569,755e4a50,208d9687,5978055e
1,5a9ed9b0,c66fca21,78171040,373c404a,25c83c98,None,8ff6f5af,0b153874,a73ee510,5ba575e7,b5a9f90e,6766a7f0,949ea585
2,68fd1e64,207b2d81,74e1a23a,9a6888fb,25c83c98,7e0ccccf,d356c7e6,5b392875,7cc72ec2,3b08e48b,727af3e2,fb8fab62,49fe3d4e
3,ae82ea21,38a947a1,1a778157,3fee2b13,25c83c98,fbad5c96,53b5f978,0b153874,a73ee510,3b08e48b,91e8fc27,4e05887a,9ff13f22
4,05db9164,2c16a946,0d427480,1b69e68d,25c83c98,7e0ccccf,ade953a9,0b153874,a73ee510,4072f40f,29e4ad33,6be9ae06,80467802
5,be589b51,38d50e09,4724f2c8,8510f416,4cf72387,7e0ccccf,38eb9cf4,0b153874,a73ee510,2462946f,7f8ffe57,f6fe1d50,46f42a63
6,05db9164,421b43cd,889a923c,29998ed1,384874ce,fe6b92e5,52283d1c,37e4aa92,a73ee510,03e48276,e51ddf94,6aaba33c,3516f6e6
7,87552397,09e68b86,d3b321f6,0327824a,4cf72387,7e0ccccf,197b4575,51d76abe,a73ee510,6c47047a,24f22538,6f697ab7,e40e52ae
8,05db9164,39dfaa0d,8f9df49c,c0c8ad07,25c83c98,None,753aa291,1f89b562,a73ee510,3b08e48b,d20ffd8f,dab8680a,6619af2b
9,be589b51,4c2bc594,d032c263,c18be181,25c83c98,7e0ccccf,7227c706,1f89b562,a73ee510,305a0646,9625b211,dfbb09fb,dccbd94b


In [8]:
pandas_df.loc[:, '_c27':'_c39']

,_c27,_c28,_c29,_c30,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39
0,07d13a8f,5182f694,f8b34416,e5ba7672,e5f8f18f,None,None,f3ddd519,None,32c7478e,b34f3128,None,None
1,1adce6ef,8736735c,59974c9c,8efede7f,1304f63b,21ddcdc9,b1252a9d,07b2853e,None,32c7478e,94bde4f2,010f6491,09b76f8d
2,b28479f6,231f3923,c6b1e1b2,2005abd1,25935396,21ddcdc9,5840adea,99c09e97,None,be7c41b4,335a6a1e,001f3601,8d8eb391
3,07d13a8f,5d76dea1,b4bb659e,776ce399,d6be9f7b,None,None,64e9eec3,None,be7c41b4,1793a828,None,None
4,b28479f6,3628a186,acfad74a,07c540c4,e4ca448c,None,None,f973405d,None,3a171ecb,9117a34a,None,None
5,b28479f6,42b3012c,ad774107,1e88c74f,582152eb,21ddcdc9,5840adea,fbaf98df,None,32c7478e,e773f0cb,001f3601,1b0ebd59
6,b28479f6,e1ac77f7,b041b04a,d4bb7bd8,2804effd,None,None,723b4dfd,None,3a171ecb,b34f3128,None,None
7,07d13a8f,36721ddc,e7287ff1,8efede7f,5aed7436,21ddcdc9,a458ea53,86dec00a,None,3a171ecb,c62cc6c2,e8b83407,33358f02
8,b28479f6,2223bbe1,ceda4ce2,776ce399,df4fffb7,21ddcdc9,5840adea,09f172ad,None,3a171ecb,c8a524a7,010f6491,4e7af834
9,8ceecbc8,7ac43a46,84898b2a,e5ba7672,bc48b783,None,None,0014c32a,ad3062eb,55dd3565,3b183c5c,None,None


Keeping only first two categorical features for simplicity

In [9]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [10]:
df = df.fillna(0, subset=num_columns) # TODO: try different default value

In [11]:
df.limit(20).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39,id
0,1,0,-1,0,0,1465,0,17,0,4,...,e5f8f18f,None,None,f3ddd519,None,32c7478e,b34f3128,None,None,12
1,0,8,0,15,20,115,24,8,23,24,...,1304f63b,21ddcdc9,b1252a9d,07b2853e,None,32c7478e,94bde4f2,010f6491,09b76f8d,39
2,0,0,5,30,4,0,0,0,5,4,...,25935396,21ddcdc9,5840adea,99c09e97,None,be7c41b4,335a6a1e,001f3601,8d8eb391,108
3,0,0,7,1,0,9128,0,0,0,7,...,d6be9f7b,None,None,64e9eec3,None,be7c41b4,1793a828,None,None,139
4,0,0,18,15,9,4494,0,0,9,8,...,e4ca448c,None,None,f973405d,None,3a171ecb,9117a34a,None,None,175
5,0,0,19,32,0,1994,0,0,19,26,...,582152eb,21ddcdc9,5840adea,fbaf98df,None,32c7478e,e773f0cb,001f3601,1b0ebd59,199
6,0,0,76,3,0,5029,0,0,16,0,...,2804effd,None,None,723b4dfd,None,3a171ecb,b34f3128,None,None,206
7,0,0,1,51,11,20692,1645,3,21,1580,...,5aed7436,21ddcdc9,a458ea53,86dec00a,None,3a171ecb,c62cc6c2,e8b83407,33358f02,235
8,0,0,19,31,8,7130,126,1,20,116,...,df4fffb7,21ddcdc9,5840adea,09f172ad,None,3a171ecb,c8a524a7,010f6491,4e7af834,252
9,0,0,2242,13,7,11786,63,29,13,87,...,bc48b783,None,None,0014c32a,ad3062eb,55dd3565,3b183c5c,None,None,268


## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm that can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm that can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import  MinMaxScaler, StringIndexer, OneHotEncoderEstimator, VectorAssembler


# in proccess
columns = cat_columns[:2]
columns_indexed = [col + '_indexed' for col in columns]
columns_vectorized = [col + '_vec' for col in columns]

vector_assembler = VectorAssembler(inputCols=num_columns + columns_vectorized, outputCol="features")
string_indexers = [StringIndexer(inputCol=col, outputCol=col + '_indexed', handleInvalid="keep") for col, col_ind in zip(columns, columns_indexed)]
one_hot_encoder = OneHotEncoderEstimator(inputCols=columns_indexed, outputCols=columns_vectorized)
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")

pipeline = Pipeline(stages=string_indexers + [one_hot_encoder] + [vector_assembler] + [scaler])

### Fit and save pipeline

In [13]:
import os
import shutil
pipeline_model = pipeline.fit(df)

path = "transforming_pipeline"
if os.path.exists(path):
    shutil.rmtree(path)
pipeline_model.save(path)
path

'transforming_pipeline'

### Load fitted pipeline

In [14]:
from pyspark.ml import PipelineModel, Pipeline

pipeline_model = PipelineModel.load(path)

### Transform dataset using pipeline

In [15]:
df_transformed = pipeline_model.transform(df).select('_c0', 'scaled_features', 'id')
df_transformed.limit(10).toPandas()

,_c0,scaled_features,id
0,1,"[0.0, 4.5314482508609755e-05, 0.0, 0.0, 0.0005...",12
1,0,"[0.0013852813852813853, 9.062896501721951e-05,...",39
2,0,"[0.0, 0.0003172013775602683, 0.000457770656900...",108
3,0,"[0.0, 0.0004078303425774878, 1.525902189669642...",139
4,0,"[0.0, 0.0009062896501721951, 0.000228885328450...",175
5,0,"[0.0, 0.0009516041326808048, 0.000488288700694...",199
6,0,"[0.0, 0.0035345296356715608, 4.577706569008926...",206
7,0,"[0.0, 0.00013594344752582924, 0.00077821011673...",235
8,0,"[0.0, 0.0009516041326808048, 0.000473029678797...",252
9,0,"[0.0, 0.10168569874932028, 0.00019836728465705...",268


### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [16]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import lit

def split_by_col(df, split_col, part_fractions):
    """
    df - DataFrame
    split_col - total order column
    part_fractions - fractions of the resulting parts
    """
    
    window = Window.orderBy(split_col).partitionBy('const')
    ranks = df \
        .select(split_col) \
        .distinct() \
        .withColumn('const', F.lit(0)) \
        .withColumn('percent_rank', F.percent_rank().over(window)) \
        .select(split_col, 'percent_rank')

    df = df.join(ranks, split_col)
    
    parts = []
    s = 0
    for part_fraction in part_fractions:
        l = s
        r = s + part_fraction
        part = df.filter((l <= F.col('percent_rank')) & (F.col('percent_rank') < r)).drop('percent_rank')
        
        parts.append(part)
        s = r
        
    return parts

In [17]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [18]:
N = df.count()

In [19]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.8, 0.09999972730867294, 0.09999972730867294)

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [ ]:
from pyspark.ml.classification import LogisticRegression


######################################
######### YOUR CODE HERE #############
######################################

In [ ]:
lr_model.transform(val_df).first()

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [22]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics


def rocauc(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass


def logloss(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass


def ne(model, df):
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    pass

In [23]:
rocauc(lr_model, val_df)

0.7031266528628317

In [24]:
ne(lr_model, val_df)

0.918952248522235

In [25]:
rocauc(lr_model, test_df)

0.7000027547230876

In [26]:
ne(lr_model, test_df)

0.9220781622206322

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [27]:
######################################
######### YOUR CODE HERE #############
######################################